In [6]:
pip install gradio gitpython rapidfuzz


In [26]:
import os
import json
import logging
from typing import List, Dict
from rapidfuzz import fuzz
import git
import gradio as gr


# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class CoALAAgent:
    def __init__(self, scripts_metadata: List[Dict]):
        """
        Initialize the agent with script metadata.
        :param scripts_metadata: List of dictionaries containing script information
        """
        self.scripts_metadata = scripts_metadata

    def interpret_query(self, query: str) -> str:
        """
        Normalize the user's query.
        :param query: User input query
        :return: Processed query string
        """
        processed_query = query.lower().strip()
        logger.info(f"Processed user query: '{processed_query}'")
        return processed_query

    def calculate_relevance(self, query: str, script: Dict) -> float:
        """
        Calculate the fuzzy match similarity for both name and description.
        :param query: User's interpreted query string
        :param script: A dictionary with script metadata
        :return: Weighted similarity score
        """
        try:
            name_similarity = fuzz.ratio(query, script["name"].lower())
            desc_similarity = fuzz.partial_ratio(query, script.get("description", "").lower())

            # Weighted similarity
            total_similarity = (name_similarity * 0.7) + (desc_similarity * 0.3)

            logger.debug(f"Calculating relevance for '{script['name']}': name_similarity={name_similarity}, desc_similarity={desc_similarity}, total_similarity={total_similarity}")
            return total_similarity
        except Exception as e:
            logger.error(f"Error computing similarity for script: {script['name']}, {e}")
            return 0.0

    def find_relevant_scripts(self, query: str) -> List[Dict]:
        """
        Dynamically find and return all relevant scripts based on the user query.
        :param query: Interpreted user query
        :return: List of relevant scripts
        """
        try:
            matches = []
            for script in self.scripts_metadata:
                # Calculate relevance score for each script
                relevance_score = self.calculate_relevance(query, script)

                # Only consider matches that are relevant enough
                if relevance_score > 60:  # Adjust the threshold if needed
                    matches.append({"relevance": relevance_score, "script": script})

            # Sort results by relevance descending
            sorted_matches = sorted(matches, key=lambda x: x["relevance"], reverse=True)

            logger.info(f"Found {len(sorted_matches)} relevant scripts.")
            return [match["script"] for match in sorted_matches]
        except Exception as e:
            logger.error(f"Error finding scripts: {e}")
            return []


def load_scripts_metadata(directory: str) -> List[Dict]:
    """
    Recursively load all scripts metadata from the directory and its subfolders.
    :param directory: Path to directory
    :return: List of script metadata dictionaries
    """
    try:
        scripts_metadata = []

        # Clone repo if not found
        if not os.path.exists(directory):
            logger.info(f"Cloning GitHub repository from {GITHUB_REPO_URL}")
            git.Repo.clone_from(GITHUB_REPO_URL, directory)

        # Walk the directory and all subdirectories recursively
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(".dyn"):
                    script_path = os.path.join(root, file)
                    script_name = os.path.splitext(file)[0]
                    scripts_metadata.append({
                        "name": script_name,
                        "path": script_path,
                        "description": f"Located at {script_path}"
                    })

        logger.info(f"Total scripts loaded: {len(scripts_metadata)}")
        return scripts_metadata
    except Exception as e:
        logger.error(f"Error loading script metadata: {e}")
        return []


# Initialize with scripts from GitHub
GITHUB_REPO_URL = "https://github.com/giobel/DynamoScripts.git"
SCRIPTS_DIR = os.path.expanduser("~") + "/dynamo_scripts"
scripts_metadata = load_scripts_metadata(SCRIPTS_DIR)
agent = CoALAAgent(scripts_metadata)


# Main Query Function
def user_query_function(user_query: str):
    """
    Handles user input query and dynamically returns dropdown options and feedback.
    """
    if not user_query:
        return [], "Please enter a valid search query."

    interpreted_query = agent.interpret_query(user_query)
    matched_scripts = agent.find_relevant_scripts(interpreted_query)

    if not matched_scripts:
        return [], "No matching scripts found."

    script_names = [script["name"] for script in matched_scripts]
    script_descriptions = [script["description"] for script in matched_scripts]

    logger.info(f"Returning {len(script_names)} scripts for dropdown.")
    return script_names, script_descriptions


def display_script_details(script_name: str, script_detail: str):
    """
    Displays the script details on user selection from the dropdown.
    """
    return f"### Selected Script\n**Name:** {script_name}\n**Details:**\n{script_detail}"


# Create the gradio interface
with gr.Blocks() as demo:
    gr.Markdown("### Search Dynamo Scripts & Select Relevant Options")

    # Input Mechanism
    query_input = gr.Textbox(label="Enter Your Search Query", placeholder="e.g., Beam Creation")
    search_button = gr.Button("Search")

    # Dropdown Output
    script_dropdown = gr.Dropdown(label="Select a Script", choices=[], interactive=True)
    script_display = gr.Textbox(label="Script Details", interactive=False)

    # Connect Search Button to Dropdown Options
    def handle_search(query):
        script_names, script_details = user_query_function(query)
        return gr.update(choices=script_names), script_names, script_details

    search_button.click(
        fn=handle_search,
        inputs=query_input,
        outputs=[script_dropdown, script_dropdown, script_display]
    )

    # On Dropdown interaction, dynamically show the script details
    script_dropdown.change(
        fn=lambda name: f"### Selected Script\n**Name:** {name}",
        inputs=script_dropdown,
        outputs=script_display
    )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4f393b899e64c6b14.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
